In [1]:
from IPython.display import display
from torchnlp.datasets import iwslt_dataset
from torchnlp.word_to_vector import GloVe
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from utils import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
vectors = GloVe()

In [3]:
train_raw = [sentence['en'] for sentence in iwslt_dataset(train=True)]
dev_raw = [sentence['en'] for sentence in iwslt_dataset(dev=True)]
test_raw = [sentence['en'] for sentence in iwslt_dataset(test=True)]
print("Train:", len(train_raw))
print("Dev:", len(dev_raw))
print("Test:", len(test_raw))

Train: 196884
Dev: 993
Test: 1305


In [4]:
print("Preprocessing the train data")
train_x, train_y = preprocess_data_trim(train_raw, vectors)
print("Preprocessing the dev data")
dev_x, dev_y = preprocess_data_trim(dev_raw, vectors)
print("Preprocessing the test data")
test_x, test_y = preprocess_data_trim(test_raw, vectors)

Preprocessing the train data
Tokenizing:



Padding:



Creating labels:



Get word vector weights



Converting labels to tensor


...


Padding:



Creating labels:



Get word vector weights



Converting labels to tensor



Printing samples of the dataset
['<pad>', '<pad>', 'When', 'I', 'was'] ('o', True)
['<pad>', 'When', 'I', 'was', 'in'] ('o', True)
['When', 'I', 'was', 'in', 'my'] ('o', False)
['I', 'was', 'in', 'my', '20s'] ('o', False)
['was', 'in', 'my', '20s', 'I'] ('o', False)
['in', 'my', '20s', 'I', 'saw'] (',', False)
['my', '20s', 'I', 'saw', 'my'] ('o', True)
['20s', 'I', 'saw', 'my', 'very'] ('o', False)
['I', 'saw', 'my', 'very', 'first'] ('o', False)
['saw', 'my', 'very', 'first', 'psychotherapy'] ('o', False)
['my', 'very', 'first', 'psychotherapy', 'client'] ('o', False)
['very', 'first', 'psychotherapy', 'client', '<pad>'] ('o', False)
['first', 'psychotherapy', 'client', '<pad>', '<pad>'] ('.', False)
['<pad>', '<pad>', 'I', 'was', 'a'] ('o', True)
['<pad>', 'I', 'was', 'a', 'Ph.D.'] ('o', False)
Total number of samples:  21760


In [5]:
from models import PuncDataset
train_loader = DataLoader(PuncDataset(train_x, train_y), batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=0)
dev_loader = DataLoader(PuncDataset(dev_x, dev_y), batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=0)
test_loader = DataLoader(PuncDataset(test_x, test_y), batch_size=BATCH_SIZE,
                         shuffle=True, num_workers=0)

In [6]:
from models import PuncLstmPersist

model = PuncLstmPersist(CLASSES, WINDOW_SIZE)
model.to(device)
criterions = [nn.CrossEntropyLoss(), nn.CrossEntropyLoss()]
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
best_model = None

In [7]:
if __name__=='__main__':
    best_model = train(train_loader, dev_loader, model, criterions, optimizer, 100, device)

    print('Finished Training')

----------------------------
Epoch: 0


Training loss for punctuation: 52.37359109262009
Training loss for capitalization: 22.714418004877896
Validation accuracy: punctuation: 94.5147%, capitalization: 98.5475%
Find a new best model!
----------------------------
Epoch: 1


Training loss for punctuation: 43.00325812715897
Training loss for capitalization: 14.442372386567333
Validation accuracy: punctuation: 94.7788%, capitalization: 98.6576%
Find a new best model!
----------------------------
Epoch: 2


Training loss for punctuation: 40.32262693985795
Training loss for capitalization: 13.136028672296757
Validation accuracy: punctuation: 95.0924%, capitalization: 98.6576%
Find a new best model!
----------------------------
Epoch: 3


Training loss for punctuation: 38.53914150020471
Training loss for capitalization: 12.163836054647387
Validation accuracy: punctuation: 94.9989%, capitalization: 98.6356%
----------------------------
Epoch: 4


...

Training loss for punctuation: 34.73434956578775
Training loss for capitalization: 10.01646379049466
Validation accuracy: punctuation: 95.1419%, capitalization: 98.575%
----------------------------
Epoch: 7


Training loss for punctuation: 33.70112561857141
Training loss for capitalization: 9.412558537782088
Validation accuracy: punctuation: 95.197%, capitalization: 98.5695%
----------------------------
Epoch: 8


Training loss for punctuation: 32.71330641417508
Training loss for capitalization: 8.86658299544901
Validation accuracy: punctuation: 95.1419%, capitalization: 98.5971%
----------------------------
Epoch: 9


Training loss for punctuation: 31.88250426104993
Training loss for capitalization: 8.380208413854655
Validation accuracy: punctuation: 95.2025%, capitalization: 98.542%
----------------------------
Epoch: 10


Training loss for punctuation: 31.06186546228676
Training loss for capitalization: 7.928368951395367
Validation accuracy: punctuation: 95.1254%, capitalization: 98.4925%
Finished Training


In [8]:
dfs = test(test_loader, best_model, device)
display(dfs[0])
display(dfs[1])

..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, po

..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, po

..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, po

Test accuracy: punctuation: 95.2895%, capitalization: 98.6029%


,predicted,predicted correctly,predicted expectation,precision,recall,f_score
punctuation,,,,,,
o,19301,18733,19042,0.970521,0.983721,0.977077
",",1151,814,1354,0.706597,0.600738,0.649382
.,1286,1171,1223,0.909868,0.956699,0.932696
?,22,17,128,0.739130,0.131783,0.223684
!,0,0,13,0.000000,0.000000,0.000000


,predicted,predicted correctly,predicted expectation,precision,recall,f_score
capitalization,,,,,,
No,19363,19130,19201,0.987916,0.996250,0.992066
Yes,2397,2326,2559,0.969975,0.908594,0.938282


In [9]:
#save_model(model)